# Scrapping  en Wikipedia Webstite 
**(fijarse si está permitido con robots.txt)**

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#Convertir a objeto e imprimir toda la pagina
soup = bs(r.content)
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"912e0aab-6d21-42ec-b96a-300095890ed0","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Walt_Disney_Pictures_films","wgTitle":"List of Walt Disney Pictures films","wgCurRevisionId":1102111985,"wgRevisionId":1102111985,"wgArticleId":1970335,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","CS1 maint: url-status","Articles wit

In [3]:
#extraer la data con el elemento italico de la pagina en css
movies = soup.select(".wikitable.sortable i")
movies[0:10]
movies[0]

<i><a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a></i>

In [4]:
def get_content_value(row_data):
  if row_data.find("li"):
    return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
  #para separar los nombres de los protagonistas
  elif row_data.find("br"):
    return [text for text in row_data.stripped_strings]
  else:
    return row_data.get_text(" ", strip=True).replace("\xa0", " ")  

def clean_tags(soup): #para limpiar referencias de algunos vinculos
  for tag in soup.find_all(["sup", "span"]):
    tag.decompose()

def get_info_box(url):

    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    clean_tags(soup)

    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
    return movie_info    


In [5]:
get_info_box("https://en.wikipedia.org/wiki/One_Little_Indian_(film)")


{'Box office': '$2 million',
 'Cinematography': 'Charles F. Wheeler',
 'Country': 'United States',
 'Directed by': 'Bernard McEveety',
 'Distributed by': 'Buena Vista Distribution',
 'Edited by': 'Robert Stafford',
 'Language': 'English',
 'Music by': 'Jerry Goldsmith',
 'Produced by': 'Winston Hibler',
 'Production company': 'Walt Disney Productions',
 'Release date': ['June 20, 1973'],
 'Running time': '90 Minutes',
 'Starring': ['James Garner',
  'Vera Miles',
  'Pat Hingle',
  "Clay O'Brien",
  'John Doucette',
  'Morgan Woodward',
  'Andrew Prine'],
 'Written by': 'Harry Spalding',
 'title': 'One Little Indian'}

In [6]:
# iterar todas las movies y adjuntarlas como lista
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies): #ojo con abusar de las request
  if index % 10 == 0:
    print(index)
  try:
    relative_path = movie['href']
    full_path = base_path + relative_path
    title = movie['title']
    
    movie_info_list.append(get_info_box(full_path))
  
  except Exception as e: #aquellas movies con nonetype (las skippeo porque son todos upcoming  o no tienen info relevante
    print(movie.get_text())
    print(e)

0
10
20
30
40
Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'find'
490
500
Elemental
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
All Night Long
'NoneType' object has no attribute 'find'
510
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
520
Shrunk
'NoneType' object has no attribute 'find'
Sister Act 3
'NoneType' object has no attribute 'find'
The Graveyard Book
'NoneType' object has no attribute 'find_all'
The Thief
'NoneType' object has no attribute 'find_all'
530
Tom Sawyer
'NoneType' object has no attribute 'find_all'
Tower of Terror


In [7]:
len(movie_info_list)

521

In [8]:
#guardarlo en formato json
import json
def save_data(title,data):
  with open(title, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [9]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [10]:
save_data("disney_data_cleaned.json", movie_info_list)

# Limpiar y convertir datos

In [11]:
movie_info_list = load_data("disney_data_cleaned.json")

### Cambiar la columna running time de str a int


In [12]:
print([movie.get('Running time', 'N/A') for movie in movie_info_list])

['41 minutes (74 minutes 1966 release)', '83 minutes', '88 minutes', '126 minutes', '74 minutes', '64 minutes', '70 minutes', '42 minutes', '70 min', '71 minutes', '75 minutes', '94 minutes', '73 minutes', '75 minutes', '82 minutes', '68 minutes', '74 minutes', '96 minutes', '75 minutes', '84 minutes', '77 minutes', '92 minutes', '69 minutes', '81 minutes', ['60 minutes (VHS and Wild Discovery version)', '71 minutes (original)'], '127 minutes', '93 minutes', '76 minutes', '75 minutes', '73 minutes', '85 minutes', '81 minutes', '70 minutes', '90 min.', '80 minutes', '75 minutes', '83 minutes', '83 minutes', '72 minutes', '97 minutes', '75 minutes', '104 minutes', '93 minutes', '105 minutes', '95 minutes', '97 minutes', '134 minutes', '69 minutes', '92 minutes', '126 minutes', '79 minutes', '97 minutes', '128 minutes', '73 minutes', '91 minutes', '105 minutes', '98 minutes', '130 minutes', '89 minutes', '93 minutes', '67 minutes', '98 minutes', '100 minutes', '118 minutes', '103 minutes'

In [13]:
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else: # si es un string
        return int(running_time.split(" ")[0])

for movie in movie_info_list:
    movie['Running time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))

In [14]:
print([movie.get('Running time (int)', 'N/A') for movie in movie_info_list])

[41, 83, 88, 126, 74, 64, 70, 42, 70, 71, 75, 94, 73, 75, 82, 68, 74, 96, 75, 84, 77, 92, 69, 81, 60, 127, 93, 76, 75, 73, 85, 81, 70, 90, 80, 75, 83, 83, 72, 97, 75, 104, 93, 105, 95, 97, 134, 69, 92, 126, 79, 97, 128, 73, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 74, 91, 91, 97, 118, 139, 131, 92, 87, 116, 93, 110, 110, 131, 101, 108, 84, 78, 75, 164, 106, 110, 99, 113, 108, 102, 85, 91, 93, 100, 100, 79, 96, 113, 89, 117, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, None, 91, 112, 115, 95, 91, 97, 104, 74, 48, 77, 104, 128, 101, 94, 104, 90, 100, 88, 93, 98, 112, 84, 97, 97, 114, 96, 97, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 90, 74, 90, 89, 110, 74, 93, 84, 83, 74, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 108, 94, 106, 102, 88, 102, 102, 97, 111, 100, 96, 96, 78, 81, 108, 89, 99, 89, 81, 92, 100, 89, 79, 91, 101, 104, 103, 86, 105, 74, 93, 92, 98, 95, 93, 87, 93, 87, 128, 77, 86, 95, 1

### Convertir columna budget y box office en floats

In [15]:
print([movie.get('Budget', 'N/A') for movie in movie_info_list])

['N/A', '$1.49 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', 'N/A', '$788,000', 'N/A', '$1.35 million', '$2.125 million', 'N/A', '$1.5 million', '$1.5 million', 'N/A', '$2.2 million', '$1,800,000', '$3 million', 'N/A', '$4 million', '$2 million', '$300,000', '$1.8 million', 'N/A', '$5 million', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$700,000', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$6 million', 'under $1 million or $1,250,000', 'N/A', '$2 million', 'N/A', 'N/A', '$2.5 million', 'N/A', 'N/A', '$4 million', '$3.6 million', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', '$4.4–6 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', '

In [16]:
import re #reggex patterns

amounts = r"thousand|million|billion"
#\d+ sirve para encontrar numbers, (,\d{3}) sirve para encontrar decimales 
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

'''
money_conversion("$12.2 million") --> 12200000 ## Word syntax
money_conversion("$790,000") --> 790000        ## Value syntax
'''
def money_conversion(money):
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]
        
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

    else:
        return None

In [17]:
for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))

### Convertir fechas en datetimes para poder utilizar inteligencia del tiempo


In [18]:
print([movie.get('Release date', 'N/A') for movie in movie_info_list])

[['May 19, 1937'], 'N/A', 'N/A', ['November 13, 1940'], ['June 27, 1941'], 'N/A', 'N/A', 'N/A', ['July 17, 1943'], 'N/A', 'N/A', 'N/A', ['September 27, 1947'], 'May 27, 1948', 'N/A', ['October 5, 1949'], 'N/A', 'N/A', 'N/A', 'N/A', ['February 5, 1953 (United States)'], ['July 23, 1953 (US)'], ['November 10, 1953'], 'N/A', ['August 17, 1954'], ['December 23, 1954'], 'May 25, 1955', ['June 22, 1955'], ['September 14, 1955'], 'December 22, 1955', 'June 8, 1956', 'July 18, 1956', ['September 4, 1956'], ['December 20, 1956'], 'June 19, 1957', 'August 28, 1957', ['December 25, 1957'], ['July 8, 1958'], ['August 12, 1958'], ['December 25, 1958'], ['January 29, 1959'], ['March 19, 1959'], 'N/A', ['November 10, 1959'], 'January 21, 1960 ( Sarasota, FL )', ['February 24, 1960'], 'May 19, 1960', 'N/A', ['November 1, 1960'], ['December 21, 1960'], ['January 25, 1961'], 'March 16, 1961', ['June 21, 1961'], ['July 12, 1961'], ['July 17, 1961'], ['December 14, 1961'], 'April 5, 1962', ['May 17, 1962'

In [19]:
from datetime import datetime

dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
        
    if date == "N/A":
        return None
        
    date_str = clean_date(date)

    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

In [20]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [21]:
#guardar el dict en pickle por la cantidad de datos
import pickle
def save_data_pickle(name, data):
  with open(name, 'wb') as f:
    pickle.dump(data, f)

  

In [22]:
import pickle
def load_data_pickle(name):
  with open(name, 'rb') as f:
     return pickle.load(f)

In [23]:
save_data_pickle("disney_movie_data_cleaned_more.pickle", movie_info_list)

In [24]:
a = load_data_pickle("disney_movie_data_cleaned_more.pickle")

# Adjuntar IMDB/Rotten Tomatoes/Metascore Api's




In [25]:
movie_info_list = load_data_pickle("disney_movie_data_cleaned_more.pickle")

In [26]:
movie_info_list[20]

{'Based on': ['Peter and Wendy', 'by', 'J. M. Barrie'],
 'Box office': '$87.4 million (United States and Canada)',
 'Box office (float)': 87400000.0,
 'Budget': '$4 million',
 'Budget (float)': 4000000.0,
 'Country': 'United States',
 'Directed by': ['Clyde Geronimi', 'Wilfred Jackson', 'Hamilton Luske'],
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Music by': 'Oliver Wallace',
 'Narrated by': 'Tom Conway',
 'Produced by': 'Walt Disney',
 'Production company': 'Walt Disney Productions',
 'Release date': ['February 5, 1953 (United States)'],
 'Release date (datetime)': datetime.datetime(1953, 2, 5, 0, 0),
 'Running time': '77 minutes',
 'Running time (int)': 77,
 'Starring': ['Bobby Driscoll',
  'Kathryn Beaumont',
  'Hans Conried',
  'Paul Collins'],
 'Story by': ['Milt Banta',
  'Bill Cottrell',
  'Winston Hibler',
  'Bill Peet',
  'Erdman Penner',
  'Joe Rinaldi',
  'Ted Sears',
  'Ralph Wright'],
 'title': 'Peter Pan'}

In [27]:
# http://www.omdbapi.com/?i=tt3896198&apikey=APIKEY usen la suya

In [28]:
import requests
import urllib
import os

def get_omdb_info(title):
    base_url = "http://www.omdbapi.com/?"
    parameters = {"apikey": "42ea2f3" , 't': title}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

get_omdb_info("into the woods")

{'Actors': 'Anna Kendrick, Meryl Streep, Chris Pine',
 'Awards': 'Nominated for 3 Oscars. 10 wins & 74 nominations total',
 'BoxOffice': '$128,002,372',
 'Country': 'United States',
 'DVD': '24 Mar 2015',
 'Director': 'Rob Marshall',
 'Genre': 'Adventure, Comedy, Drama',
 'Language': 'English',
 'Metascore': '69',
 'Plot': 'A witch tasks a childless baker and his wife with procuring magical items from classic fairy tales to reverse the curse put on their family tree.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTY4MzQ4OTY3NF5BMl5BanBnXkFtZTgwNjM5MDI3MjE@._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'PG',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '71%'},
  {'Source': 'Metacritic', 'Value': '69/100'}],
 'Released': '25 Dec 2014',
 'Response': 'True',
 'Runtime': '125 min',
 'Title': 'Into the Woods',
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'James Lapine',
 'Year': '2014',
 'imdbID': 'tt2180411',
 '

In [29]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['Rated'] = omdb_info.get('Rated', None)
    movie['Awards'] = omdb_info.get('Awards', None)
    movie['Genre'] = omdb_info.get('Genre', None)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)

In [30]:
movie_info_list[-60]

{'Awards': 'Nominated for 1 Oscar. 4 wins & 65 nominations total',
 'Box office': '$141.9 million',
 'Box office (float)': 141900000.0,
 'Budget': '$175–200 million',
 'Budget (float)': 175000000.0,
 'Cinematography': ['Sharon Calahan', 'Adam Habib'],
 'Country': 'United States',
 'Directed by': 'Dan Scanlon',
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Edited by': 'Catherine Apple',
 'Genre': 'Animation, Adventure, Comedy',
 'Language': 'English',
 'Music by': ['Mychael Danna', 'Jeff Danna'],
 'Produced by': 'Kori Rae',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Rated': 'PG',
 'Release date (datetime)': None,
 'Release dates': ['February 21, 2020 ( Berlinale )',
  'March 6, 2020 (United States)'],
 'Running time': '102 minutes',
 'Running time (int)': 102,
 'Screenplay by': ['Dan Scanlon', 'Jason Headley', 'Keith Bunin'],
 'Starring': ['Tom Holland',
  'Chris Pratt',
  'Julia Louis-Dreyfus',
  'Octavia Spencer'],
 'Story by': [

In [ ]:
#convertir los score en float
for movie in movie_info_list:
    movie['imdb'] = float(movie['imdb'])
    movie['metascore'] = float(movie['metascore'])
    movie['rotten_tomatoes'] = float(movie['rotten_tomatoes'].strip('%'))

In [34]:
save_data_pickle("disney_movie_data_final.pickle", movie_info_list)

### Transformando el diccionario en csv con la librería pandas


In [35]:
movie_info_copy = [movie.copy() for movie in movie_info_list]


In [36]:
#removiendo los strings extras de datetime
for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [37]:
save_data("disney_data_final.json", movie_info_copy)

In [41]:
#guardando csv
import pandas as pd

df = pd.DataFrame(movie_info_list)
df.tail(10)

,title,Production company,Distributed by,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),...,Simplified,Original title,Suggested by,Layouts by,Original concept by,Music,Lyrics,Book,Basis,Productions
511,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Stephen Flaherty,Lynn Ahrens,Lynn Ahrens,"[My Love, My Love; or, The Peasant Girl, by, R...","[1990, Broadway, 1994 UK/Europe, 1995, West En..."
512,Robin Hood,Walt Disney Productions,Buena Vista Distribution,"[November 8, 1973]",83 minutes,United States,English,$33 million,83.0,5000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,Snow White,NaN,"[Walt Disney Studios, Motion Pictures]",NaN,NaN,United States,English,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,SpaceCamp,ABC Motion Pictures,20th Century Fox,"[June 6, 1986 (United States)]",107 minutes,United States,English,"$9,697,739 (USA)",107.0,18000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,William Finn,William Finn,Rachel Sheinkin,"[C-R-E-P-U-S-C-U-L-E, by Rebecca Feldman]","[2004 Workshop, 2005 Off-Broadway, 2005 Broadw..."
516,The Aristocats,Walt Disney Productions,Buena Vista Distribution,NaN,79 minutes,United States,English,$191 million,79.0,4000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,The Sword in the Stone,Walt Disney Productions,Buena Vista Distribution,NaN,74 minutes,United States,English,$22.2 million (United States and Canada),74.0,3000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,Tinker Bell .mw-parser-output .nobold{font-wei...,DisneyToon Studios,"[Walt Disney Studios, Home Entertainment]",NaN,[468 minutes],United States,English,NaN,468.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,The Rocketeer,NaN,Buena Vista Pictures Distribution,"[June 21, 1991]",108 minutes,United States,English,$46.7 million (USA),108.0,35000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520,Young Woman and the Sea,NaN,Disney+,NaN,NaN,United States,English,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.to_csv("disney_movie_data_final.csv")